## Import packages and libraries

In [ ]:
import cv2, os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.gridspec as gridspec

%matplotlib inline

### Define functions for loading images

In [ ]:
# Collect image file paths
def original_image_paths ():
    folder = 'images/original'
    files = os.listdir(folder)
    files.sort()
    files = ['{}/{}'.format(folder, file) for file in files]
    return files

# Load the images into a numpy array
def np_read_images(img_file_paths):
    X_data = []

    for index, file_path in enumerate(img_file_paths):
        img = mpimg.imread(file_path) 
        X_data.append(img)

#     X_data = np.array(X_data, dtype = np.float32) # Convert to numpy
    return X_data

In [ ]:
# Display the vehicles in a grid
vehicle_paths = original_image_paths()
vehicle_images = np_read_images(vehicle_paths)

gs = gridspec.GridSpec(3, 2)
gs.update(wspace = 0.30, hspace = 0.30)

fig, ax = plt.subplots(figsize = (20, 20))
# Front Passenger
plt.subplot(gs[0])
plt.imshow(vehicle_images[0])
plt.title('Front Passenger')
# Rear Passenger
plt.subplot(gs[1])
plt.imshow(vehicle_images[1])
plt.title('Rear Passenger')
# Rear Center
plt.subplot(gs[2])
plt.imshow(vehicle_images[2])
plt.title('Rear Center')
# Rear Driver
plt.subplot(gs[3])
plt.imshow(vehicle_images[3])
plt.title('Rear Driver')
# Front Driver
plt.subplot(gs[4])
plt.imshow(vehicle_images[4])
plt.title('Front Driver')
# Front Driver
plt.subplot(gs[5])
plt.imshow(vehicle_images[5])
plt.title('Front Center')
plt.show()

## OpenCV Image Analysis

In [ ]:
print(cv2.__version__)

### Image Gradients - Sobel and Scharr, Laplacian Derivatives

In [ ]:
# Front Passenger

img = cv2.imread('./images/original/1C4PJLCX8KD22313106.jpeg',0)

laplacian = cv2.Laplacian(img,cv2.CV_64F)
sobelx = cv2.Sobel(img,cv2.CV_64F,1,0, ksize=31)
sobely = cv2.Sobel(img,cv2.CV_64F,0,1, ksize=31)

gs = gridspec.GridSpec(2, 2)
gs.update(wspace = 0.10, hspace = 0.10)

fig, ax = plt.subplots(figsize = (16, 16))

plt.subplot(gs[0])
plt.imshow(img, cmap = 'gray')
plt.title('Original')
plt.subplot(gs[1])
plt.imshow(laplacian, cmap = 'gray')
plt.title('Laplacian')
plt.subplot(gs[2])
plt.imshow(sobelx, cmap = 'gray')
plt.title('Sobel X')
plt.subplot(gs[3])
plt.imshow(sobely, cmap = 'viridis')
plt.title('Sobel Y')

plt.show()

### Canny Edge Detection
OpenCV puts all the above in single function, cv2.Canny(). We will see how to use it. First argument is our input image. Second and third arguments are our minVal and maxVal respectively. Third argument is aperture_size. It is the size of Sobel kernel used for find image gradients. By default it is 3. Last argument is L2gradient which specifies the equation for finding gradient magnitude. If it is True, it uses the equation mentioned above which is more accurate, otherwise it uses this 

In [ ]:
img = cv2.imread('./images/original/1C4PJLCX8KD22313105.jpeg')
edges = cv2.Canny(img, 800, 400, True)
fig, ax = plt.subplots(figsize = (16, 16))
plt.subplot(121),plt.imshow(img,cmap = 'gray')
plt.title('Original Image'), plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(edges,cmap = 'gray')
plt.title('Edge Image'), plt.xticks([]), plt.yticks([])

plt.show()

### Interactive Foreground Extraction using GrabCut Algorithm

[GrabCut algorithm](http://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_grabcut/py_grabcut.html#grabcut)

Now we go for grabcut algorithm with OpenCV. OpenCV has the function, cv2.grabCut() for this. We will see its arguments first:

- *img* - Input image
- *mask* - It is a mask image where we specify which areas are background, foreground or probable background/foreground etc. It is done by the following flags, **cv2.GC_BGD, cv2.GC_FGD, cv2.GC_PR_BGD, cv2.GC_PR_FGD**, or simply pass 0,1,2,3 to image.
- *rect* - It is the coordinates of a rectangle which includes the foreground object in the format (x,y,w,h)
- *bdgModel, fgdModel* - These are arrays used by the algorithm internally. You just create two np.float64 type zero arrays of size (1,65).
- *iterCount* - Number of iterations the algorithm should run.
- *mode* - It should be **cv2.GC_INIT_WITH_RECT** or **cv2.GC_INIT_WITH_MASK** or combined which decides whether we are drawing rectangle or final touchup strokes.


In [ ]:
img = cv2.imread('./images/original/1C4PJLCX8KD22313106.jpeg')
mask = np.zeros(img.shape[:2],np.uint8)

bgdModel = np.zeros((1,65),np.float64)
fgdModel = np.zeros((1,65),np.float64)

rect = (450,40,800,900)
cv2.grabCut(img, mask, rect, bgdModel, fgdModel, 5, cv2.GC_INIT_WITH_RECT)

mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')
img = img*mask2[:,:,np.newaxis]

fig, ax = plt.subplots(figsize = (10, 10))

plt.imshow(img),plt.show()

In [ ]:
import cv2
cv2.saliency

In [ ]:
image = cv2.imread('./images/original/1C4PJLCX8KD22313105.jpeg')
saliency = cv2.saliency.StaticSaliencySpectralResidual_create()
(success, saliencyMap) = saliency.computeSaliency(image)
saliencyMap = (saliencyMap * 255).astype("uint8")
plt.imshow(saliencyMap)
plt.show()

In [ ]:
# initialize OpenCV's static fine grained saliency detector and
# compute the saliency map
saliency = cv2.saliency.StaticSaliencyFineGrained_create()
(success, saliencyMap) = saliency.computeSaliency(image)
 
# if we would like a *binary* map that we could process for contours,
# compute convex hull's, extract bounding boxes, etc., we can
# additionally threshold the saliency map
threshMap = cv2.threshold(saliencyMap, 0, 255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
 
gs = gridspec.GridSpec(2, 2)
gs.update(wspace = 0.10, hspace = 0.10)

fig, ax = plt.subplots(figsize = (16, 16))

plt.subplot(gs[0])
plt.imshow(image, cmap = 'gray')
plt.title('Original')
plt.subplot(gs[1])
plt.imshow(saliencyMap)
plt.title('saliencyMap')
plt.subplot(gs[2])
plt.imshow(threshMap, cmap = 'gray')
plt.title('Thresh')

## scikit-image

In [ ]:
from skimage.feature import hog
from skimage import data, exposure

In [ ]:
# The original image is inverted as the object must be white.
image = cv2.imread('./images/original/1C4PJLCX8KD22313105.jpeg')

fd, hog_image = hog(image, orientations=8, pixels_per_cell=(16, 16),
                    cells_per_block=(1, 1), visualize=True, multichannel=True)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (16, 6), sharex=True, sharey=True)

ax1.axis('off')
ax1.imshow(image, cmap=plt.cm.gray)
ax1.set_title('Input image')

# Rescale histogram for better display
hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))

ax2.axis('off')
ax2.imshow(hog_image_rescaled, cmap=plt.cm.gray)
ax2.set_title('Histogram of Oriented Gradients')
plt.show()


In [ ]:
from skimage import io as skio
url = './images/original/1C4PJLCX8KD22313105.jpeg'
img = skio.imread(url, flatten=True)

In [ ]:
print("shape of image: {}".format(img.shape))
print("dtype of image: {}".format(img.dtype))

In [ ]:
from skimage import filters
sobel = filters.sobel(img)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.rcParams['figure.dpi'] = 200

In [ ]:
plt.imshow(sobel)

In [ ]:
blurred = filters.gaussian(sobel, sigma=2.0)
plt.imshow(blurred)

In [ ]:
import cv2
 
# Read the images
foreground = cv2.imread("./images/original/1C4PJLCX8KD22313105.jpeg")
background = cv2.imread("./images/background.png")
alpha = cv2.imread("./images/alpha/1C4PJLCX8KD22313105.png")
 
# Convert uint8 to float
foreground = foreground.astype(float)
background = background.astype(float)
 
# Normalize the alpha mask to keep intensity between 0 and 1
alpha = alpha.astype(float)/255
 
# Multiply the foreground with the alpha matte
foreground = cv2.multiply(alpha, foreground)
 
# Multiply the background with ( 1 - alpha )
background = cv2.multiply(1.0 - alpha, background)
 
# Add the masked foreground and background.
outImage = cv2.add(foreground, background)
 

%matplotlib inline
plt.rcParams['figure.dpi'] = 200
plt.imshow(outImage/255)